<a href="https://colab.research.google.com/github/Philipsd21/HebrewOCR/blob/main/OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test 1

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install opencv-contrib-python
!pip3 install cropyble
!apt-get install tesseract-ocr-heb
!tesseract --list-langs
!pip3 install openpyxl
!pip install roman

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import pytesseract

folderPath = "/content/drive/MyDrive/Jastrow/Hebrewbooks_org_38236.zip (Unzipped Files)"
#folderPath = "/content/Test_Data/"
myRevList = os.listdir(folderPath)
print(len(myRevList))

In [ ]:
import cv2

In [ ]:
from openpyxl import load_workbook
from pathlib import Path
def get_headers():
  path = Path('/content','drive','MyDrive','Copy of Headers.xlsx')
  print(path)
  wb = load_workbook(path)
  sheet = wb.active
  for row in sheet.iter_rows():
    for cell in row:
        value = cell.value
        if value is None:
          continue
        else:
          if len(value) == 1:
            print(value)

get_headers()

In [ ]:
def search(word, to_search_in):
  closest_word = difflib.get_close_matches(word, [], n=2)

In [ ]:
from pytesseract.pytesseract import image_to_boxes
from google.colab.patches import cv2_imshow
import difflib
import codecs
import cv2
completed_runs = 0
run = 0
corpus = []
Pages = {}
def threshold_test(image1):

 
  # path to input image is specified and 
  # image is loaded with imread command  
  # cv2.cvtColor is applied over the
  # image input with applied parameters
  # to convert the image in grayscale
  img = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
  ret, thresh = cv2.threshold(img, 120, 255, cv2.THRESH_TOZERO)


  return thresh

def detect_words(img):
  header_image = img[0:100, 0:1400]
  left_image = header_image[0:100,200:474]
  center_image = header_image[0:100, 475:925]
  right_image = header_image[0:100, 925:1200]
  return header_image, left_image, center_image, right_image

def print_detected(thresholded):
  for image in list(detect_words(thresholded))[1:]:
    cv2_imshow(image)

number_for_testing = 200
pytesseract.pytesseract.tesseract_cmd = r"/usr/bin/tesseract"
found_first_page, first_page_information = False, []
for images in myRevList:
    img = cv2.imread(f'{folderPath}/{images}')
    if img is None: corpus.append("Could not read the image.")
    else:
        run += 1
        thresholded = threshold_test(img)
        header_img, left_img, center_img, right_img = detect_words(thresholded)
                
        
        center_ans = (pytesseract.image_to_string(center_img,lang='eng'))
        left_ans = (pytesseract.image_to_string(left_img,lang='heb'))
        right_ans = (pytesseract.image_to_string(right_img,lang='heb'))
        answers = [center_ans, left_ans, right_ans]

        for i in range(3): answers[i] = ''.join([character for character in answers[i] if character.isalnum()])
        if (any([answer == '' in answer for answer in answers]) and not found_first_page):
          continue
        elif not found_first_page: first_page_information, found_first_page = [answers[0], run], True
        completed_runs += 1
        if completed_runs > number_for_testing:  break
        #If the page number has returned an empty string, base it off of the previous pages number
        if answers[0] == '':  answers[0] = (run - first_page_information[1]) + int(first_page_information[0])
        
        #Saving the pages information Pages[Page Number] = 
        Pages[answers[0]] = [answers[1], answers[2]]

        print(f'{run}: {images}\tLeft:{answers[1]}\tCenter:{answers[0]}\tRight:{answers[2]}')
        #print_detected(thresholded)
        cv2_imshow(detect_words(thresholded)[0])



        corpus.append(0)

print(Pages)

In [ ]:
import shutil


def write_to_file(image_name, text):
  if not os.path.exists('Text Files'):
    os.makedirs('Text Files')
  file = codecs.open('Text Files/'+str(image_name) + '.txt', "w+", "utf-8")
  file.write(','.join(text))
  file.close()

def clear_files():
    shutil.rmtree('Text Files')
    os.makedirs('Text Files')

clear_files()
for i in list(Pages.keys()):
  write_to_file(str(i),Pages[i])


# Test 2

In [ ]:
def get_page_number(word,words_dictionary):
  word_list = words_dictionary[word[0]]
  print(word_list)


In [ ]:
# A function to create the headers from the incomplete database
# Only somewhat acurate

from openpyxl import load_workbook
from pathlib import Path

words = {}
all_words = []
letters = []
everything = []
pages = {}
def set_letters():
  wb = load_workbook(Path('/content','drive','MyDrive','Copy of Headers.xlsx'))
  sheet = wb.active
  for row in sheet.iter_rows():
    for cell in row:
        value = cell.value
        if value is None:
          continue
        else:
          if len(value) == 1:
            words[value] = []
            letters.append(value)
          else:
            all_words.append(value)
          everything.append(value)
  for word in all_words:
    print(word)
    words[word[0]].append(word)
set_letters()
print(letters)
is_on_new_page = {
    'א':True
, 'ב':False
, 'ג':False
, 'ד':False
, 'ה':False
, 'ו':False
, 'ז':False
, 'ח':False
, 'ט':False
, 'י':False
, 'כ':False
, 'ל':True
, 'מ':False
, 'נ':False
, 'ס':True
, 'ע':False
, 'פ':False
, 'צ':False
, 'ק':False
, 'ר':False
, 'ש':True
, 'ת':False
    

}

print(is_on_new_page['ת'])

In [ ]:
def set_pages():
  page = 1
  to_add = []
  for word in everything:
    if len(word) == 1 and is_on_new_page[word]:
      if len(to_add) != 0:
        pages[page] = to_add
        to_add = []
        page += 1
      pages[page] = word
      page += 1
      continue
    to_add.append(word)      
    if len(to_add) == 2:          
      pages[page] = to_add
      to_add = []
      page += 1
set_pages()
print(pages)
print(len(pages))

In [ ]:
# A function to find the minimum and maximum page that a word would how up in
# word: A String of the word to be found
# word_list: A dictionary of each starting word sorted by starting letter ['ת': ['תאוטני', 'תבה'
def get_boundary_pages(word,word_list):
  start_number = 0
  for letter in word_list.keys():
    if letter == word[0]:
      break
    start_number += len(word_list[letter])
  start_number += list(word_list.keys()).index(word[0])

  end_number = 0
  next_letter = letters[letters.index(word[0]) + 1]
  print(next_letter)
  for letter in word_list.keys():
    if letters[letters.index(letter)] == next_letter:
      break
    end_number += len(word_list[letter])
  end_number += list(word_list.keys()).index(next_letter)
  return start_number, end_number

print(words)
print(get_boundary_pages('ריסר',words))

# Test 3


In [ ]:
from openpyxl import load_workbook
from pathlib import Path
searchable = []
wb = load_workbook(Path('/content','drive','MyDrive','Copy of Headers.xlsx'))
sheet = wb.active
for row in sheet.iter_rows():
  for cell in row:
    if cell.value is not None:
      searchable.append(cell.value) 
print(len(searchable))

In [ ]:
import requests
import roman
word = input('enter your word:')

for i in range (0,3):
  r = requests.get(f'https://www.sefaria.org/Jastrow_אָמַר_{roman.toRoman(i)}')
  print(r.content)